<img src="Images/BERT_diagrams.pdf" width="1000">

## Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('smile-annotations-final.csv', names = ['id', 'text', 'category'])
df.set_index('id', inplace = True)
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [3]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

**Removing the noncode, and double category emtions**

In [4]:
df = df[~df.category.str.contains('\|')] ## removing all the data that has (|) operator in  category column 

In [5]:
df = df[df.category != 'nocode']

In [6]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

**Lets encode the category column like happy --> 0 , not-relevant --> 1 etc**

In [7]:
possible_lables = df.category.unique()

In [8]:
## Creating lable dictionary
label_dict = {}

for index, lables in enumerate(possible_lables):
    label_dict[lables] = index
    

In [9]:
label_dict # This is how we can create a dictionary of all the possible lables

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

**Creating the label columns as per the dictionary**

In [10]:
df['label'] =df.category.map(label_dict)

In [11]:
df.tail()

,text,category,label
id,,,
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0
615246897670922240,@MrStuchbery @britishmuseum Mesmerising.,happy,0
613016084371914753,@NationalGallery The 2nd GENOCIDE against #Bia...,not-relevant,1


## Training/Validation Split

In [12]:
from sklearn.model_selection import train_test_split

**Since data is nor symmetric hence we use stratfied technique of splitting. where data is splitted equally for each category.**

In [13]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.label.values, 
                                                 test_size=0.15,
                                                 random_state=17,
                                                 stratify=df.label.values)

In [14]:
y_train

array([0, 0, 0, ..., 0, 0, 2], dtype=int64)

**Lests make a new column in the dataframe which stores the information that weather a particular row is used for training or validation**

In [15]:
df['data_type'] = ['not_set']*df.shape[0]

In [16]:
df.loc[X_train, 'data_type'] = 'train'  ## Replacing the 
df.loc[X_val, 'data_type'] = 'val'

In [17]:
df.groupby(['category', 'label', 'data_type']).count() ## confirms that the data is equally splitted as per the category

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## Loading Tokenizer and Encoding our Data

In [18]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case= True)

In [20]:
encoded_data_train = tokenizer.batch_encode_plus(df[df.data_type== 'train'].text.values, add_special_tokens=True,
                                                return_attention_mask=True,
                                                pad_to_max_length = True,
                                                max_length=256,
                                                return_tensors ='pt')

encoded_data_val = tokenizer.batch_encode_plus(df[df.data_type== 'val'].text.values, add_special_tokens=True,
                                                return_attention_mask=True,
                                                pad_to_max_length = True,
                                                max_length=256,
                                                return_tensors ='pt')
## encode_plus return the dictionary

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\katiy\anaconda3\envs\tensorflow_gpu\lib\site-packages\transformers\tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [22]:
len(dataset_val)

223

## Setting up BERT Pretrained Model

In [23]:
from transformers import BertForSequenceClassification

In [24]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # select the GPU if available

In [49]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels= len(possible_lables),
    output_attentions = False,
    output_hidden_states = False
)
model = model.to(device) # to force the model to run on the GPU
device

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

device(type='cuda', index=0)

## Creating Data Loaders

In [26]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [27]:
batch_size = 4 ## usually we take higher batch size such as 32

data_loader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size = batch_size
)

data_loader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size = 32
) ## Since val data set is quite small hence we are choosing the higher batch size

## Setting Up Optimizer and Scheduler

In [28]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [29]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, ## recomended is 2e-5 to 5e-5
    eps = 1e-8
)

C:\Users\katiy\anaconda3\envs\tensorflow_gpu\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps= 0,
    num_training_steps= len(data_loader_train)*epochs
)

##  Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [31]:
import numpy as np

In [32]:
from sklearn.metrics import f1_score

In [33]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [34]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k, v in label_dict.items()}  ## changes the key into value and value into key
    
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    
    for labels in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == labels]
        y_true = labels_flat[labels_flat == labels]
        print(f'Class: {label_dict_inverse[labels]}')
        print(f'Accuracy: {len(y_preds[y_preds==labels])}/{len(y_true)}\n')

## Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [35]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [36]:
torch.cuda.is_available()

True

In [37]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }


        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [38]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train = 0
    progress_bar = tqdm(data_loader_train,
                       desc = 'Epoch {:1d}'.format(epoch),
                       leave  = False,
                       disable = False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            
            'input_ids'        : batch[0],
            'attention_mask'  : batch[1],
            'labels'          : batch[2]
            
        }
        
        outputs = model(**inputs)
        
        loss  = outputs[0]
        loss_train_total = loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'trainig_loss': '{:.3f}'. format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')
    tqdm.write(f'\n Epoch {epoch}')
    loss_train_avg = loss_train_total/len(data_loader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    val_loss, predictions, true_vals = evaluate(data_loader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 score: {val_f1}')
    

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/315 [00:00<?, ?it/s]


 Epoch 1
Training loss: 7.8649026533914e-05
Validation loss: 0.5681339076587132
F1 score: 0.8246606034722624


Epoch 2:   0%|          | 0/315 [00:00<?, ?it/s]


 Epoch 2
Training loss: 0.004472180775233678
Validation loss: 0.6534616862024579
F1 score: 0.8524540621143191


...


 Epoch 8
Training loss: 2.1383646757356706e-06
Validation loss: 0.7598000509398324
F1 score: 0.8721358633080866


Epoch 9:   0%|          | 0/315 [00:00<?, ?it/s]


 Epoch 9
Training loss: 4.4127172302632106e-06
Validation loss: 0.752623758145741
F1 score: 0.8693771608433505


Epoch 10:   0%|          | 0/315 [00:00<?, ?it/s]


 Epoch 10
Training loss: 1.5077168225414224e-06
Validation loss: 0.7486688494682312
F1 score: 0.8726514462604477


## Loading and Evaluating our Model

In [45]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [50]:
model.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [51]:
model.load_state_dict(
    torch.load('Models/BERT_ft_epoch10.model',
               map_location=torch.device('cpu')
              )
)

<All keys matched successfully>

In [52]:
_, predictions, true_vals = evaluate(data_loader_val)

In [53]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 164/171

Class: not-relevant
Accuracy: 21/32

Class: angry
Accuracy: 7/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 2/5

Class: surprise
Accuracy: 2/5

